# Quick Start

Below is a sample demo of interaction with the environment.

## Build environment & get static information from the environment

Build an environment and show facilities & units in this environment. Demo configuration file path: https://github.com/microsoft/maro/tree/master/maro/simulator/scenarios/supply_chain/topologies/example/config.yml

Here is a illustration of this example "world":
![Illustration of the world](img/example_world.png)

In [1]:
# import necessary packages
import numpy as np
from pprint import pprint

In [2]:
from maro.simulator import Env

# Initialize an Env for Supply Chain scenario. 
env = Env(scenario="supply_chain", topology="example", start_tick=0, durations=100)

In [3]:
print(env._business_engine.world._facility_name2id_mapping)

{'Supplier_001': 1, 'Supplier_002': 8, 'Warehouse_001': 17, 'Retailer_001': 28}


`env.summary` contains rich static information about the environment. For future convenience, we first parse `env.summary` to make two "name-id" mappings for facilities and units, so that we could acquire an entity's id through its name.

In [4]:
from example_utils import get_facility_name2id_mapping, get_unit_name2id_mapping

facility_name2id = get_facility_name2id_mapping(env)
unit_name2id = get_unit_name2id_mapping(env)

print("*** Facility name-id mapping ***")
pprint(facility_name2id)
print()
print("*** Unit name-id mapping ***")
pprint(unit_name2id)

*** Facility name-id mapping ***
{'Retailer_001': 28, 'Supplier_001': 1, 'Supplier_002': 8, 'Warehouse_001': 17}

*** Unit name-id mapping ***
{'Retailer_001__consumer1': 31,
 'Retailer_001__consumer2': 37,
 'Retailer_001__consumer3': 34,
 'Retailer_001__products1': 30,
 'Retailer_001__products2': 36,
 'Retailer_001__products3': 33,
 'Retailer_001__seller1': 32,
 'Retailer_001__seller2': 38,
 'Retailer_001__seller3': 35,
 'Retailer_001__storage': 29,
 'Supplier_001__distribution': 3,
 'Supplier_001__manufacture3': 7,
 'Supplier_001__products3': 6,
 'Supplier_001__storage': 2,
 'Supplier_002__distribution': 10,
 'Supplier_002__manufacture1': 14,
 'Supplier_002__manufacture2': 16,
 'Supplier_002__products1': 13,
 'Supplier_002__products2': 15,
 'Supplier_002__storage': 9,
 'Warehouse_001__consumer1': 23,
 'Warehouse_001__consumer2': 25,
 'Warehouse_001__consumer3': 27,
 'Warehouse_001__distribution': 19,
 'Warehouse_001__products1': 22,
 'Warehouse_001__products2': 24,
 'Warehouse_001__p

## Get dynamic information from the environment

In [5]:
# Import helper function for showing environment status from example_utils.py
# This helper function is just to make the output of this notebook more beautiful. 
# You can design the format of the information display according to your own preferences.
from example_utils import show_status

env.step(None)  # Tick 0. Activate the enviornment, do not send any action
show_status(env)

*** Env status after tick 0 ***
  Storage summary:
    [Facility]      [remain] / [capacity]
    Supplier_001        9900 /      10000
    Supplier_002        9800 /      10000
    Warehouse_001      27000 /      30000
    Retailer_001       17880 /      20000
  Storage detail:
    [Facility]       [sku1] [sku2] [sku3]
    Supplier_001          0      0    100
    Supplier_002        100    100      0
    Warehouse_001      1000   1000   1000
    Retailer_001        698    702    720
  Manufacture status:
    [Facility]      
    Supplier_001    sku_id = 3, manufacturing = 0, unit cost = 1
    Supplier_002    sku_id = 1, manufacturing = 0, unit cost = 1
    Supplier_002    sku_id = 2, manufacturing = 0, unit cost = 0


According to the configuration, there might be initial stock in storage, so the remaining space might **NOT** be equal to the total capacity at the beginning. For example, `Warehouse_001` contains 1000 initial stock for each SKU, so the gap between the remaining space and the total capacity is 1000\*3=3000.

The number of products and remaining space of `Retailer_001` is a random number and you may get different results **with different random seeds**. That is because the initial stocks in `Retailer_001` are read from the topology configuration and therefore are fixed, but some of the stocks will be sold in the first tick according to the demand sampled from a Gamma distribution. In other words, the number of sold products is a random variable, so the remaining stock and the remaining space therefore is also random.

Since we have not sent any manufacturing action to the environment, the manufacturing number of all facilities should be zero.

## Send actions to the environment

Actions in the Supply Chain scenario are simple. There are only two types of actions: `ManufactureAction` and `ConsumerAction`. `ManufactureAction` refers to the instruction for the facility to start production, and `ConsumerAction` refers to the action that one facility purchases some SKUs from other facilities.

### Manufacture action

Let `Supplier_001` produce 1000 products of SKU 3. The remaining space of `Supplier_001` after executing this action should be 9900-1000=8900, and the number of SKU 3 becomes 100+1000=1100. In addition, the manufacturing number of `Supplier_001 ~ SKU 3` is 1000.

There is not production speed limit for facilities, so the production will be done within this tick. 

We could notice that the remaining stock after the first tick and the second tick are different, because sales are happening all the time, so the remaining stock will change accordingly in every tick. You will see similar situations in the following ticks.

In [6]:
from maro.simulator.scenarios.supply_chain import ManufactureAction
action = ManufactureAction(id=unit_name2id["Supplier_001__manufacture3"], production_rate=1000)  # Supplier_001 produces 1000 products of SKU 3
env.step([action])  # Tick 1
show_status(env)

*** Env status after tick 1 ***
  Storage summary:
    [Facility]      [remain] / [capacity]
    Supplier_001        8900 /      10000
    Supplier_002        9800 /      10000
    Warehouse_001      27000 /      30000
    Retailer_001       18182 /      20000
  Storage detail:
    [Facility]       [sku1] [sku2] [sku3]
    Supplier_001          0      0   1100
    Supplier_002        100    100      0
    Warehouse_001      1000   1000   1000
    Retailer_001        597    596    625
  Manufacture status:
    [Facility]      
    Supplier_001    sku_id = 3, manufacturing = 1000, unit cost = 1
    Supplier_002    sku_id = 1, manufacturing = 0, unit cost = 1
    Supplier_002    sku_id = 2, manufacturing = 0, unit cost = 0


Then let `Supplier_001` produce 9000 products of SKU 3. This time, however, `Supplier_001` does not have enough remaining space, so only 8900 products are produced.

In [7]:
action = ManufactureAction(id=unit_name2id["Supplier_001__manufacture3"], production_rate=9000)  # Supplier_001 produces 9000 products of SKU 3
env.step([action])  # Tick 2
show_status(env)

*** Env status after tick 2 ***
  Storage summary:
    [Facility]      [remain] / [capacity]
    Supplier_001           0 /      10000
    Supplier_002        9800 /      10000
    Warehouse_001      27000 /      30000
    Retailer_001       18467 /      20000
  Storage detail:
    [Facility]       [sku1] [sku2] [sku3]
    Supplier_001          0      0  10000
    Supplier_002        100    100      0
    Warehouse_001      1000   1000   1000
    Retailer_001        510    495    528
  Manufacture status:
    [Facility]      
    Supplier_001    sku_id = 3, manufacturing = 8900, unit cost = 1
    Supplier_002    sku_id = 1, manufacturing = 0, unit cost = 1
    Supplier_002    sku_id = 2, manufacturing = 0, unit cost = 0


Now we let `Supplier_002` produce 7000 products of SKU 1. However, as we can see in the environment status after tick 3, the increment of SKU 1 is 4900, not the 7000 we expected. This is because there is a hidden limitation for supplier storage: **if a supplier could produce multiple types of product, each type of product will share the storage space evenly**. `Supplier_002` could produce SKU 1 and SKU 2, so each type of product could only use half of the storage space, which is 10000/2=5000. Therefore, we could produce SKU 1 up to 5000 even if there is spare storage space.

In [8]:
action = ManufactureAction(id=unit_name2id["Supplier_002__manufacture1"], production_rate=7000)  # Supplier_002 produces 7000 products of SKU 1
env.step([action])  # tick 3
show_status(env)

*** Env status after tick 3 ***
  Storage summary:
    [Facility]      [remain] / [capacity]
    Supplier_001           0 /      10000
    Supplier_002        4900 /      10000
    Warehouse_001      27000 /      30000
    Retailer_001       18769 /      20000
  Storage detail:
    [Facility]       [sku1] [sku2] [sku3]
    Supplier_001          0      0  10000
    Supplier_002       5000    100      0
    Warehouse_001      1000   1000   1000
    Retailer_001        403    395    433
  Manufacture status:
    [Facility]      
    Supplier_001    sku_id = 3, manufacturing = 0, unit cost = 1
    Supplier_002    sku_id = 1, manufacturing = 4900, unit cost = 1
    Supplier_002    sku_id = 2, manufacturing = 0, unit cost = 0


### Consumer action

We now try to send 8000 products of SKU 3 from `Supplier_001` to `Warehouse_001`. The production number of SKU 3 in `Supplier_001` has not been changed in tick 4 because the `ConsumerAction` will NOT be activated immediately. Instead, it will be activated in the next tick.

In [9]:
from maro.simulator.scenarios.supply_chain import ConsumerAction
action = ConsumerAction(id=unit_name2id["Warehouse_001__consumer3"], product_id=3, source_id=facility_name2id["Supplier_001"], 
                        quantity=8000, 
                        vlt=4,  # Velocity of the vehicle. 
                                # The number of steps of the vehicle will be automatically calculated according to vlt and path length.
                                # In the current case, it is 12 / 4 = 3 steps.
                        reward_discount=0)
env.step([action])  # Tick 4
show_status(env)

*** Env status after tick 4 ***
  Storage summary:
    [Facility]      [remain] / [capacity]
    Supplier_001           0 /      10000
    Supplier_002        4900 /      10000
    Warehouse_001      27000 /      30000
    Retailer_001       19064 /      20000
  Storage detail:
    [Facility]       [sku1] [sku2] [sku3]
    Supplier_001          0      0  10000
    Supplier_002       5000    100      0
    Warehouse_001      1000   1000   1000
    Retailer_001        307    295    334
  Manufacture status:
    [Facility]      
    Supplier_001    sku_id = 3, manufacturing = 0, unit cost = 1
    Supplier_002    sku_id = 1, manufacturing = 0, unit cost = 1
    Supplier_002    sku_id = 2, manufacturing = 0, unit cost = 0


The `ConsumerAction` is activated in tick 5. `Supplier_001` assigns the order to one vehicle, the vehicle hence loads 8000 products in tick 5, so we can see the product number of SKU 3 in `Supplier_001` decreases by 8000. Then, the vehicle shall take its first step at tick 6. Since the expected number of arrival ticks is 3, this order should arrive after tick 8 (ticks on road: tick 6, tick 7, and tick 8).

In [10]:
env.step(None)  # Tick 5
show_status(env)

*** Env status after tick 5 ***
  Storage summary:
    [Facility]      [remain] / [capacity]
    Supplier_001        8000 /      10000
    Supplier_002        4900 /      10000
    Warehouse_001      27000 /      30000
    Retailer_001       19394 /      20000
  Storage detail:
    [Facility]       [sku1] [sku2] [sku3]
    Supplier_001          0      0   2000
    Supplier_002       5000    100      0
    Warehouse_001      1000   1000   1000
    Retailer_001        194    179    233
  Manufacture status:
    [Facility]      
    Supplier_001    sku_id = 3, manufacturing = 0, unit cost = 1
    Supplier_002    sku_id = 1, manufacturing = 0, unit cost = 1
    Supplier_002    sku_id = 2, manufacturing = 0, unit cost = 0


Transition status can be found in `env.metrics`.

In [11]:
# Helper function
def show_warehouse_001_incoming_orders() -> None:
    # Incoming orders of Warehouse_001 by SKU
    print(f"*** Incoming order summary of Warehouse_001 after tick {env.tick} ***")
    pending_order = env.metrics["facilities"][facility_name2id["Warehouse_001"]]["in_transit_orders"]
    for sku_id, nproduct in pending_order.items():
        print(f"Incoming number of SKU {sku_id}: {nproduct}")

    # Incoming SKU 3 orders of Warehouse_001 by day
    pending_order_daily = env.metrics["products"][unit_name2id["Warehouse_001__products3"]]["pending_order_daily"].tolist()
    for day, nproduct in enumerate(pending_order_daily, 1):
        print(f"Number of SKU 3 products arrive UID_W001_PROD3 in {day} days: {nproduct}")

show_warehouse_001_incoming_orders()

*** Incoming order summary of Warehouse_001 after tick 5 ***
Incoming number of SKU 1: 0
Incoming number of SKU 2: 0
Incoming number of SKU 3: 8000
Number of SKU 3 products arrive UID_W001_PROD3 in 1 days: 0
Number of SKU 3 products arrive UID_W001_PROD3 in 2 days: 0
Number of SKU 3 products arrive UID_W001_PROD3 in 3 days: 8000
Number of SKU 3 products arrive UID_W001_PROD3 in 4 days: 0


We run an empty tick (do not execute any action), and then we can see the change of daily pending orders.

In [12]:
env.step(None)  # Tick 6
show_warehouse_001_incoming_orders()

*** Incoming order summary of Warehouse_001 after tick 6 ***
Incoming number of SKU 1: 0
Incoming number of SKU 2: 0
Incoming number of SKU 3: 8000
Number of SKU 3 products arrive UID_W001_PROD3 in 1 days: 0
Number of SKU 3 products arrive UID_W001_PROD3 in 2 days: 8000
Number of SKU 3 products arrive UID_W001_PROD3 in 3 days: 0
Number of SKU 3 products arrive UID_W001_PROD3 in 4 days: 0


After tick 8, the daily pending orders of `Warehouse_001` are cleared, but the storage of `Warehouse_001` has not been changed. This is because the vehicle needs one extra tick to unload products.

In [13]:
env.step(None)  # Tick 7
env.step(None)  # Tick 8
show_status(env)
print()
show_warehouse_001_incoming_orders()

*** Env status after tick 8 ***
  Storage summary:
    [Facility]      [remain] / [capacity]
    Supplier_001        8000 /      10000
    Supplier_002        4900 /      10000
    Warehouse_001      27000 /      30000
    Retailer_001       20000 /      20000
  Storage detail:
    [Facility]       [sku1] [sku2] [sku3]
    Supplier_001          0      0   2000
    Supplier_002       5000    100      0
    Warehouse_001      1000   1000   1000
    Retailer_001          0      0      0
  Manufacture status:
    [Facility]      
    Supplier_001    sku_id = 3, manufacturing = 0, unit cost = 1
    Supplier_002    sku_id = 1, manufacturing = 0, unit cost = 1
    Supplier_002    sku_id = 2, manufacturing = 0, unit cost = 0

*** Incoming order summary of Warehouse_001 after tick 8 ***
Incoming number of SKU 1: 0
Incoming number of SKU 2: 0
Incoming number of SKU 3: 8000
Number of SKU 3 products arrive UID_W001_PROD3 in 1 days: 0
Number of SKU 3 products arrive UID_W001_PROD3 in 2 days: 0
Numb

After tick 9, the number of products of SKU 3 in `Warehouse_001` becomes 1000+8000=9000. Once the unloading is done, the vehicle will return to the source facility within the same tick (in this case, within tick 9). In other words, we omit the time consumption of the return trip.

In [14]:
env.step(None)  # Tick 9
show_status(env)

*** Env status after tick 9 ***
  Storage summary:
    [Facility]      [remain] / [capacity]
    Supplier_001        8000 /      10000
    Supplier_002        4900 /      10000
    Warehouse_001      19000 /      30000
    Retailer_001       20000 /      20000
  Storage detail:
    [Facility]       [sku1] [sku2] [sku3]
    Supplier_001          0      0   2000
    Supplier_002       5000    100      0
    Warehouse_001      1000   1000   9000
    Retailer_001          0      0      0
  Manufacture status:
    [Facility]      
    Supplier_001    sku_id = 3, manufacturing = 0, unit cost = 1
    Supplier_002    sku_id = 1, manufacturing = 0, unit cost = 1
    Supplier_002    sku_id = 2, manufacturing = 0, unit cost = 0


In the above example, there are no obstacles to the transportation of the order. The actual situation might be much more complicated, for example:
- The product in the source facility is insufficient.
- The remaining storage space of the target facility is insufficient.
- There are no available vehicles in the source facility.

You could mock several actions to trigger these situations and see what will happen. Make your hands dirty and enjoy :)

## Break the black box

As we have mentioned, `env.summary`, `env.snapshot_list`, and `env.metrics` contain pretty rich information about the environment. Due to the limitation of this notebook, we cannot go through every detail and you may explore what else you can find on your own. If there is still other information you need that cannot be got from these interfaces, you can always "hack" into the `env.business_engine.world` and get the facilities/units by calling `get_entity()`. Then, you can directly access, or even control these facilities/units.

In [15]:
supplier_001 = env.business_engine.world.get_entity(facility_name2id["Supplier_001"])
pprint({
    "facility_id": supplier_001.id,
    "facility_name": supplier_001.name,
    "facility_products": supplier_001.products,
    "facility_storage": supplier_001.storage,
    "facility_distribution": supplier_001.distribution,
    "facility_location": (supplier_001.x, supplier_001.y)
})

{'facility_distribution': <maro.simulator.scenarios.supply_chain.units.distribution.DistributionUnit object at 0x000001EEAADD7B48>,
 'facility_id': 1,
 'facility_location': (0, 0),
 'facility_name': 'Supplier_001',
 'facility_products': {3: <maro.simulator.scenarios.supply_chain.units.product.ProductUnit object at 0x000001EEBA85F648>},
 'facility_storage': <maro.simulator.scenarios.supply_chain.units.storage.StorageUnit object at 0x000001EEBA84FD48>}
